#  Project: Leukemia trials study on ClinicalTrials.gov
## extract each trials' nct_id, phase, start_date, completion_date, lead_sponser, address
1. plotthe tendency curves for  Leukemia(completed, terminated, active);

2. download all cancer data, allow users to plot the tendency curves for certain disease and certain phase;

3. practise data structure and unit test;


In [1]:
from bs4 import BeautifulSoup
import lxml
import os
import glob
import pandas as pd
import numpy as np
from datetime import date
import matplotlib.pyplot as plt
import time
import re

In [2]:
#infile = open("./Leukemia_completed/NCT01188681.xml","r")
#infile = open("./Leukemia_all/NCT01188681.xml","r")
NCTtest="NCT00001575"
#NCTtest="NCT01188681"
tic = time.clock()
infile = open("./clinicaltrialgov_all/"+NCTtest+".xml","r")
contents = infile.read()
soup = BeautifulSoup(contents,'xml')
nct_id = soup.find_all('id_info')[0].find_all('nct_id')
nct_title = soup.find_all('brief_title')
phase = soup.find_all('phase')
start_date = soup.find_all('start_date')
completion_date = soup.find_all('completion_date')
lead_sponser=soup.find_all('sponsors')[0].find_all('lead_sponsor')[0].find_all('agency')
#extract drug infomation
intervention_type=soup.find_all('intervention')[0].find_all('intervention_type')
intervention_name=soup.find_all('intervention')[0].find_all('intervention_name')
contact_name=soup.find_all('point_of_contact')[0].find_all('name_or_title')
contact_org=soup.find_all('point_of_contact')[0].find_all('organization')
contact_phone=soup.find_all('point_of_contact')[0].find_all('phone')
contact_email=soup.find_all('point_of_contact')[0].find_all('email')
#address=soup.find_all('location')[0].find_all('address')
toc = time.clock()
print('processing time: ', toc-tic, 'sec')


print('NCTID: ', nct_id[0].get_text())
print('NCT_title: ', nct_title[0].get_text())
print('start date: ', start_date[0].get_text())
print('completion date: ', completion_date[0].get_text())
print('lead sponser: ', lead_sponser[0].get_text())
print('intervention type: ', intervention_type[0].get_text())
print('phase: ', phase[0].get_text())
print('intervention name: ', intervention_name[0].get_text())
print('contact_name: ', contact_name[0].get_text())
print('contact_org: ', contact_org[0].get_text())
print('contact_phone: ', contact_phone[0].get_text())
print('contact_email: ', contact_email[0].get_text())
#intervention: drug, biological, radiation, procedure, behavior, device, others
#overall_status: completed, recruiting, terminated;
# after extract the disease NCT, the function should be splitted: extract drug; extract time; extract recruiting
#print('address: ',address[0].get_text())


processing time:  0.062382999999999855 sec
NCTID:  NCT00001575
NCT_title:  Anti-Tac(90 Y-HAT) to Treat Hodgkin's Disease, Non-Hodgkin's Lymphoma and Lymphoid Leukemia
start date:  April 1997
completion date:  November 2013
lead sponser:  National Cancer Institute (NCI)
intervention type:  Biological
phase:  Phase 1/Phase 2
intervention name:  Y-90 Humanized Anti-Tac
contact_name:  Thomas Waldmann M.D.
contact_org:  National Cancer Institute
contact_phone:  301-496-6653
contact_email:  waldmann@nih.hhs.gov


## trial class should include:
NCTID

phase

start_date

completion_date

lead_sponser

drugname may only available for some trials

In [3]:
#search NCTID for Disease, and overall_status
#there are over 1000 cancers...
tic = time.clock()
Tstatuslist=[]
Cresultlist=[]
nct_idlist0=[]
nct_titlelist0=[]
phaselist0=[]
start_datelist0=[]
intervention_typelist0=[]
intervention_namelist0=[]

#path='./Leukemia_all/'
#path='./clinicaltrialgov_all/'  #the real deal!!!
path='./Leukemia_completed/'
for filename in glob.glob(os.path.join(path, '*.xml')):
    infile = open(filename,"r")
    contents = infile.read()
    soup = BeautifulSoup(contents,'xml') 
    nct_id = soup.find_all('id_info')[0].find_all('nct_id')[0].get_text()
    Tstatus = soup.find_all('overall_status')[0].get_text()
    nct_title = soup.find_all('brief_title')[0].get_text()
    phase = soup.find_all('phase')[0].get_text()
    #clinical result is available or not
    if len(soup.find_all('clinical_results'))>0:
        Cresult = 1
    else:
        Cresult = 0
    
    if len(soup.find_all('start_date'))>0:
        start_date = soup.find_all('start_date')[0].get_text()
    else:
        start_date = None
    if len(soup.find_all('intervention'))>0:
        intervention_type=soup.find_all('intervention')[0].find_all('intervention_type')[0].get_text()
    else:
        intervention_type=None
        
    if len(soup.find_all('intervention'))>0:
        intervention_name=soup.find_all('intervention')[0].find_all('intervention_name')[0].get_text()
    else:
        intervention_name=None
        
    
    nct_idlist0.append(nct_id)
    nct_titlelist0.append(nct_title)
    Tstatuslist.append(Tstatus)
    Cresultlist.append(Cresult)
    phaselist0.append(phase)
    start_datelist0.append(start_date)
    intervention_typelist0.append(intervention_type)
    intervention_namelist0.append(intervention_name)

toc = time.clock()
print('processing time: ', toc-tic, 'sec')


processing time:  39.25119 sec


In [4]:
dfID1= pd.DataFrame(index=range(0,len(nct_idlist0)), columns=['nct_id','trial_status','clinical_result',\
                                                             'phase','start_date','intervention_type',\
                                                              'intervention_name','nct_title'])
#add disease names then it is done!
dfID1.nct_id=nct_idlist0;
dfID1.trial_status=Tstatuslist;
dfID1.clinical_result=Cresultlist;
dfID1.phase=phaselist0;
dfID1.start_date=start_datelist0;
dfID1.intervention_type=intervention_typelist0;
dfID1.intervention_name=intervention_namelist0;
dfID1.nct_title=nct_titlelist0;
dfID1.head(10)

,nct_id,trial_status,clinical_result,phase,start_date,intervention_type,intervention_name,nct_title
0,NCT00001575,Completed,1,Phase 1/Phase 2,April 1997,Biological,Y-90 Humanized Anti-Tac,"Anti-Tac(90 Y-HAT) to Treat Hodgkin's Disease,..."
1,NCT00001586,Completed,1,Phase 2,September 1997,Biological,Rituximab,Treatment of Chronic Lymphocytic Leukemia/Smal...
2,NCT00001941,Completed,1,Phase 1/Phase 2,December 1999,Biological,daclizumab,Anti-Tac for Treatment of Leukemia
3,NCT00002766,Completed,1,Phase 3,March 1996,Biological,dactinomycin,Comparison of Two Combination Chemotherapy Reg...
4,NCT00003659,Completed,1,Phase 2,September 1998,Biological,filgrastim,"Fludarabine, Cyclophosphamide, and Rituximab i..."
5,NCT00006237,Completed,1,Phase 3,August 2000,Biological,interleukin-2,S0008: Chemotherapy Plus Biological Therapy in...
6,NCT00006392,Completed,1,Phase 3,July 2001,Drug,Vitamin E,S0000 Selenium and Vitamin E in Preventing Pro...
7,NCT00014495,Completed,1,Phase 1/Phase 2,November 2000,Biological,filgrastim,Chemotherapy and Monoclonal Antibody Therapy i...
8,NCT00025662,Completed,1,Phase 2,May 2001,Drug,RFT5-SMPT-dgA,Selective T-Cell Depletion to Reduce GVHD (Pat...
9,NCT00027560,Completed,1,Phase 2,July 2001,Biological,alemtuzumab,"Melphalan, Fludarabine, and Alemtuzumab Follow..."


In [18]:
dfID1.to_csv('./clinicaltrialgov_all/scrape_all.csv')

In [5]:
# the dataframe for recruiting trails contact info
tic = time.clock()
nct_idlist=[]
contact_namelist=[]
contact_orglist=[]
contact_phonelist=[]
contact_emaillist=[]
path='./Leukemia_completed/'
#path='./Leukemia_recruiting/'
#path='./clinicaltrialgov_all/' #the real deal
for filename in glob.glob(os.path.join(path, '*.xml')):
    infile = open(filename,"r")
    contents = infile.read()
    soup = BeautifulSoup(contents,'xml') 
    nct_id = soup.find_all('id_info')[0].find_all('nct_id')[0].get_text()
    if len(soup.find_all('point_of_contact'))>0:
        if len(soup.find_all('point_of_contact')[0].find_all('name_or_title'))>0:
            contact_name=soup.find_all('point_of_contact')[0].find_all('name_or_title')[0].get_text()
        else:
            contact_name=None
        if len(soup.find_all('point_of_contact')[0].find_all('organization'))>0:
            contact_org=soup.find_all('point_of_contact')[0].find_all('organization')[0].get_text()
        else:
            contact_org=None
        if len(soup.find_all('point_of_contact')[0].find_all('phone'))>0:
            contact_phone=soup.find_all('point_of_contact')[0].find_all('phone')[0].get_text()
        else:
            contact_phone=None
        if len(soup.find_all('point_of_contact')[0].find_all('email'))>0:
            contact_email=soup.find_all('point_of_contact')[0].find_all('email')[0].get_text()
        else:
            contact_email=None
    elif len(soup.find_all('overall_contact'))>0:
        if len(soup.find_all('overall_contact')[0].find_all('last_name'))>0:
            contact_name=soup.find_all('overall_contact')[0].find_all('last_name')[0].get_text()
        else:
            contact_name=None
        if len(soup.find_all('overall_official'))>0:
            if len(soup.find_all('overall_official')[0].find_all('affiliation'))>0:
                contact_org=soup.find_all('overall_official')[0].find_all('affiliation')[0].get_text()
            else:
                contact_org=None
        else:
            contact_org=None
        if len(soup.find_all('overall_contact')[0].find_all('phone'))>0:
            contact_phone=soup.find_all('overall_contact')[0].find_all('phone')[0].get_text()
        else:
            contact_phone=None
        if len(soup.find_all('overall_contact')[0].find_all('email'))>0:
            contact_email=soup.find_all('overall_contact')[0].find_all('email')[0].get_text()
        else:
            contact_email=None
    else:
        contact_name=None
        contact_org=None
        contact_phone=None
        contact_email=None
    nct_idlist.append(nct_id)
    contact_namelist.append(contact_name)
    contact_orglist.append(contact_org)
    contact_phonelist.append(contact_phone)
    contact_emaillist.append(contact_email)
toc = time.clock()
print('processing time: ', toc-tic, 'sec')

processing time:  38.08584999999999 sec


In [6]:
dfID2= pd.DataFrame(index=range(0,len(nct_idlist)), columns=['nct_id','contact_name','contact_org',\
                                                             'contact_phone','contact_email'])
dfID2.nct_id=nct_idlist;
dfID2.contact_name=contact_namelist;
dfID2.contact_org=contact_orglist;
dfID2.contact_phone=contact_phonelist;
dfID2.contact_email=contact_emaillist;
dfID2.head(10)

,nct_id,contact_name,contact_org,contact_phone,contact_email
0,NCT00001575,Thomas Waldmann M.D.,National Cancer Institute,301-496-6653,waldmann@nih.hhs.gov
1,NCT00001586,"Wyndham H. Wilson, M.D.","National Cancer Institute, National Institutes...",301-435-2415,wilsonw@mail.nih.gov
2,NCT00001941,"Thomas Waldmann, M.D.","National Cancer Institute, National Institues ...",301-496-6653,tawald@mail.nih.gov
3,NCT00002766,Dr. Peter Maslak,Memorial Sloan Kettering Cancer Center,212-639-5518,maslakp@mskcc.org
4,NCT00003659,Dr. David Scheinberg,Memorial-Sloan Kettering Cancer Center,646-888-2190,scheinbd@mskcc.org
5,NCT00006237,SWOG Melanoma Statistician,SWOG statistical office,206-667-4408,None
6,NCT00006392,Study Statistician,SWOG Statistical Center,2066674623,None
7,NCT00014495,Dr. Martin Tallman,Memorial Sloan Kettering Cancer Center,212-639-3842,tallmanm@mskcc.org
8,NCT00025662,A. John Barrett,NHLBI,301-402-4170,barrettjj@mail.nih.gov
9,NCT00027560,Dr. Hugo Castro-Malaspina,Memorial Sloan-Kettering Cancer Center,212-639-8197,castro-h@mskcc.org


In [34]:
dfID2.to_csv('./clinicaltrialgov_all/scrape_contact.csv')